### csv 파일 출처
* [나이스 교육정보 개방 포털](https://open.neis.go.kr/portal/data/service/selectServicePage.do?page=1&rows=10&sortColumn=&sortDirection=&infId=OPEN13920190311110530306647&infSeq=1#none)

In [134]:
import requests
import json

# 위도 경도 구하기 ---- kako api
def get_coordinate(addr):
    url = 'https://dapi.kakao.com/v2/local/search/address.json?query={address}'.format(address=addr)
    headers = {"Authorization": "KakaoAK " + 'cba6eb29be7c48e97d9a5cd246429cdd'}
    result = json.loads(str(requests.get(url, headers=headers).text))
    match_first = result['documents'][0]['address']
    return float(match_first['x']), float(match_first['y'])

### 행정구역_지도
![행정구역_지도](../img/%ED%96%89%EC%A0%95%EA%B5%AC%EC%97%AD_%EC%A7%80%EB%8F%84.png)
* [출처 | 위키백과](https://ko.wikipedia.org/wiki/%EB%8C%80%ED%95%9C%EB%AF%BC%EA%B5%AD%EC%9D%98_%ED%96%89%EC%A0%95_%EA%B5%AC%EC%97%AD)

In [135]:
# 광역시, 특별시, 특별자치시, 특별자치도 - 다른 행정구역에 소속되지 않음 ( 광역시와 행정구역을 분리하여 생각 )

# 1특별시, 6광역시, 1특별자치시, 8도, 1특별자치도
# => 17개의 데이터 파일 생성 

# html 이름은 csv 파일의 '교육청'빼고 설정

In [141]:
# 학교 정보 크롤링(네이버)
import requests
from bs4 import BeautifulSoup

def crawling_high_school_data(high_school_name) -> list:

    url = f'https://search.naver.com/search.naver?where=nexearch&sm=top_hty&fbm=1&ie=utf8&query={high_school_name}'
 
    response = requests.get(url)

    html = response.text

    soup = BeautifulSoup( html, 'html.parser' )

    # 0 : 주소
    # 1 : 전화번호
    # 2 : 개교 연도
    # 3 : 학생수
    a_tag = soup.select('.home') # 홈페이지 주소
    data  = soup.select('dd') # 주소, 전화번호, 개교 연도, 학생수 등

    # 리스트에 담음
    list_data = []
    for i in data[:4]:
        list_data.append(i.text)

    # 홈페이지 주소 추가
    link = []
    for i in a_tag:
        link.append(i.attrs['href'])
    
    try:
        list_data.append(link[0])
    except:
        print(high_school_name)
        return

    # 0 : 주소
    # 1 : 전화번호
    # 2 : 개교 연도
    # 3 : 학생수
    # 4 : 홈페이지
    return list_data
    

In [142]:
crawling_high_school_data('대정고등학교')

['제주특별자치도 서귀포시 대정읍 일주서로2498번길 21',
 ' 064-730-0100 ',
 '1952년 06월 02일',
 '293명 남 293명, 여 0명 ',
 'http://daejeong.jje.hs.kr']

In [150]:
import pandas as pd

high_schools_data = {}

error = []

def put_high_school_data(high_school_name, affiliations):
    high_school_data = crawling_high_school_data(high_school_name)
    
    # index
    # 0 : 주소
    # 1 : 전화번호
    # 2 : 개교 연도
    # 3 : 학생수
    # 4 : 홈페이지
    try:
        address = high_school_data[0]
        phone = high_school_data[1].strip(' ')
        birth = high_school_data[2]

        # index
        # 0 : 총 학생수
        # 2 : 남 학생수
        # 4 : 여 학생수
        students_data = high_school_data[3].split(' ')

        # 0 : 전체 학생수 / 2 : 남학생 수 / 4 : 여학생 수
        if len(students_data) <= 2: # 하지만 split한 list의 길이가 2 이하이면 총 학생수만
            total_students = students_data[0]
            boys            = '0명'
            girls           = '0명'
        else:
            total_students = students_data[0]
            boys            = students_data[2].strip(',')
            girls           = students_data[4]

        # 경도, 위도
        longitude, latitude = get_coordinate(high_school_data[0])
    except:
        error.append(high_school_name)
        return
    
    link = high_school_data[4]
    high_schools_data[high_school_name] = {
        '주소' : address,
        '위도' : latitude,
        '경도' : longitude,
        '전화번호' : phone,
        '재학생 수' : total_students,
        '남학생 수' : boys,
        '여학생 수' : girls,
        '홈페이지' : link,
        '계열' : affiliations
    }

    

In [151]:
put_high_school_data('대정고등학교', '일반계')

In [153]:

# 딕셔너리에 집어넣고 csv 파일 만듦

import os
import pandas as pd

path = '../code/교육청_csv/'

file_list = os.listdir(path) # 폴더 안 파일 목록 가져옴

offices_of_education_csv = file_list

for i in offices_of_education_csv:
    path = '../code/교육청_csv/'
    
    df = pd.read_csv(path + i)

    high_school_names = list(set(df['학교명']))

    for j in high_school_names:
        # 학교 계열
        _list = list(df[df['학교명'] == j]['계열명'])
        affiliations = ''
        for k in _list:
            affiliations += k + ' '
            if _list[-1] != k:
                affiliations += ', '

        put_high_school_data(j, affiliations) 

    high_school_dataframe = pd.DataFrame(high_schools_data).T
    csv_name = i.strip('교육청.csv') + '고등학교데이터.csv'
    path = '../data_csv/' + csv_name
    high_school_dataframe.to_csv(path)


춘천여자고등학교부설방송통신고등학교
고한여자종합고등학교
설악고등학교부설방송통신고등학교
양양여자고등학교
함백여자고등학교
학력인정 한국YMCA원주고등학교
학력인정 한국YMCA원주고등학교(야간)
하장고등학교
원주고등학교부설방송통신고등학교
강릉제일고등학교부설방송통신고등학교
학력인정 강릉인문중고등학교
묵호고등학교부설방송통신고등학교
학력인정 강원산업중고등학교


FileNotFoundError: [Errno 2] No such file or directory: '../data_csv/강원도고등학교데이터.csv경기도교육청.csv'

In [ ]:
# 로고 주소 공식
# 홈페이지 주소 + / + img src
